In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore", category=np.ComplexWarning)
np.random.seed(140)

We will continue the study of Brownian motion, but now in the context of matrices. First let's consider symmetric matrices where the entries are determined by a brownian motion process. To simulate such a process, we need to generate symmetric matrices with standard normal.

In [2]:
def normal_symmetric(n):
    A = np.random.normal(size=(n,n))
    A -= np.diag(A.diagonal())
    return (A + A.T)/np.sqrt(2) + np.diag(np.random.normal(size=n))

# 2x2 Symmetric Matrices

Next we will choose $t_{\text{max}}$ such that we can run our process from $0 \leq t \leq t_{\text{max}}$

In [3]:
t_max = 100

Next we generate a 2x2 symmetric standard normal matrices that will represent the displacements of our process through time. 

In [4]:
matrix_displacement = [normal_symmetric(2) for time in range(t_max)]

Similar to regular Brownian Motion, the process can be created by taking the cumulative sum of our displacement values.

In [5]:
Bmat_path = np.cumsum(matrix_displacement, axis=0)

We can now calculate the eigenvalues of the process at every time.

In [6]:
Bmat_eigs = np.linalg.eigvals(Bmat_path)
Bmat_eigs.sort()

We can visualize how to eigenvalues behave on the real axis (recall we are currently working with symmetric matrices, so the eigenvalues must be real).

In [7]:
fig, ax = plt.subplots()
ax.set_xlim(-50,50)
ax.set_ylim(-1,1)
plt.xlabel("Re")
plt.ylabel("Im")
plt.title("Process of Eigenvalues")
plt.plot([-50, 50], [0, 0], linewidth=1, c="black")
eig1, = ax.plot(Bmat_eigs[0][0],0, marker="o")
eig2, = ax.plot(Bmat_eigs[0][1],0, marker="o")

def update(t):
    eig1.set_data([Bmat_eigs[t][0]],[0])
    eig2.set_data([Bmat_eigs[t][1]],[0])
    return (eig1,eig2)


eig_animation = animation.FuncAnimation(fig, update, interval=50, blit=True, repeat=True,
                    frames=100)
plt.close()
HTML(eig_animation.to_html5_video())

We can visualize a line plot of the movement of these eigenvalues parameterized by time.

In [8]:
fig, ax = plt.subplots()
ax.set_xlim(0,50)
ax.set_ylim(-50,50)
line1, = ax.plot([], [], lw=2)
line2, = ax.plot([], [], lw=2)
plt.xlabel("t")
plt.ylabel("Eigenvalues")
plt.title("Process of Eigenvalues")
eig1 = [max(B) for B in Bmat_eigs]
eig2 = [min(B) for B in Bmat_eigs]


def eig_path(time):
    line1.set_data(np.arange(time),  eig1[:time])
    line2.set_data(np.arange(time),  eig2[:time])
    return (line1, line2)

plt.close()

In [9]:
eig_animation = animation.FuncAnimation(fig, eig_path, interval=50, blit=True, repeat=True,
                    frames=100)
HTML(eig_animation.to_html5_video())

The motion of these eigenvalues are stochastic processes that satisfy the following Stochastic Differential Equation

$$d\lambda_i = dB_i + \sum_{1 \leq j \leq n: j \neq i} \frac{dt}{\lambda_i - \lambda_j} $$

# 5x5 Symmetric Matrices

We can run the above process for matrices of any size. Below we plot the movement of eigenvalues of the process for 5x5 matrices.

In [10]:
n = 5
matrix_5_displacement = [normal_symmetric(n) for time in range(t_max)]
Bmat_5_path = np.cumsum(matrix_5_displacement, axis=0)
Bmat_5_eigs = np.linalg.eigvals(Bmat_5_path)
Bmat_5_eigs.sort()

In [11]:
fig, ax = plt.subplots()
ax.set_xlim(0,50)
ax.set_ylim(-50,50)
plt.xlabel("t")
plt.ylabel("Eigenvalues")
plt.title("Process of Eigenvalues")
lines = [ax.plot([], [], lw=2)[0] for i in range(n)]


def eig_path(time):
    for i in range(n):
        current_line = lines[i]
        current_line.set_data(np.arange(time),  Bmat_5_eigs.T[i][:time])
    return lines

plt.close()

In [12]:
eig_n_animation = animation.FuncAnimation(fig, eig_path, interval=50, blit=True, repeat=True,
                    frames=100)
HTML(eig_n_animation.to_html5_video())

# Standard Normal Matrices

So far we have only seen symmetric matrices, which means we have only worked with real eigenvalues. If instead all the entries of our matrix was generated by independent Brownian motions, we would expect many of our eigenvalues to be complex. We have chosen to demonstrate this example using 100x100 matrices. We begin by creating our displacement matrices.

In [13]:
n = 100
matrix_100_displacement = [np.random.normal(size=(n,n)) for time in range(t_max)]

We compute the process by taking the cumulative sum again and finding the eigenvalues.

In [14]:
Bmat_100_path = np.cumsum(matrix_100_displacement, axis=0)
Bmat_100_eigs = np.linalg.eigvals(Bmat_100_path)

We can animate the eigenvalues of the process in the Complex plane. 

In [15]:
fig, ax = plt.subplots()
ax.set_xlim(-100,100)
ax.set_ylim(-100,100)
plt.xlabel("Re")
plt.ylabel("Im")
plt.title("Process of Eigenvalues")
eig_plots = [ax.plot(Bmat_100_eigs[0][i],0, marker="o", c="black")[0] for i in range(n)]

def update(t):
    for i in range(n):
        eig_plots[i].set_data([np.real(Bmat_100_eigs[t][i])],[np.imag(Bmat_100_eigs[t][i])])
    return eig_plots

plt.close()

In [16]:
multiple_eigs_animation = animation.FuncAnimation(fig, update, interval=100, blit=True, repeat=True,
                    frames=100)
HTML(multiple_eigs_animation.to_html5_video())

Although we now have many complex eigenvalues, we can see that at all time there is still several real ones. It turns out that we can expect around $\sqrt{n}$ real eigenvalues when running through this procedure for $n \times n$ matrices.